In [1]:
import pandas as pd
import numpy as np

## (a) Select a 2-class dataset with continuous 1D features.

Read iris.data, modify data to 1D 2-class

In [2]:
import pandas as pd
def rf(fn):
    data = pd.read_csv(fn, sep=',', header=None, skipinitialspace=True)
    ncol= len(data.columns)-1
    mdata = data.loc[0:99, [3, 4]]
#     x = mdata.ix[:,:ncol-1]
#     y = mdata.ix[:,ncol]
#     print x[3].head()
#     print y.tail()
#     return x,y
    return mdata

fn = 'iris.data'
data = rf(fn)
print data.tail()

### Tail of the dataset shows as above.

## (b) Estimate the model parameters and compute a discriminant function based on the distribution in each class.

In [85]:
# cls is whole dataset, cnum is #of classes in dataset. eg cnum = 2 in q1 q2, cnum = n in q3
import numpy as np
from pylab import *
def getPrmtr(train_data,cnum):
    m = np.zeros(cnum)
    mean = np.zeros(cnum)
    sigma = np.zeros(cnum)
    for i in range(0, cnum):
        subcls = train_data[train_data[4] == i+1]
        x=subcls[3]
        m[i] = len(x)
        mean[i] = x.mean()
        for ex in x:
            sigma[i] += (ex-mean[i])**2
    return (mean, sigma/mean, m )

fn = 'iris.data'
data = rf(fn)
temp = getPrmtr(data,2)
print 'Means of two classes:'
print temp[0]
print 'Sigmas of two classes'
print temp[1]

Means of two classes:
[ 0.244  1.326]
Sigmas of two classes
[ 2.30819672  1.44509804]


## (c) Classify the examples and measure your error. Compute the confusion matrix, precision, recall, F-measure, and accuracy.

In [166]:
# K fold
def k_fold(size,k, shuffle = False):
    index = np.arange(0,size) 
    if shuffle:
        np.random.shuffle(index)
    index = np.reshape(index, (k,size/k))
#     print index
    test = []     
    train = []
    cursor = 0
    for i in range(0,k):
        test.append(index[i])
        temp = np.delete(index,i,0)
        train.append(temp.flatten())
    return test, train

In [74]:
# calculate the membership for one example x
def calcm(x, a, m, s):
    g= -0.5*((x-m)**2)/(s**2)-np.log(s)/2+np.log(a)
    return g

In [145]:
# given a test dataset value, return a list of argmax g 
def classificaiton(data_test,al,ml,sl):
    x_test = data_test[3]
    prelist = []
    for ex in x_test:
        c1 = membership = calcm(ex,al[0],ml[0],sl[0])
        c2 = membership = calcm(ex,al[1],ml[1],sl[1])
        if c1<c2:
            prelist.append(2)
        else:
            prelist.append(1)
    return prelist

In [181]:
# get confusion matrix
def cMatrix(testdata,prelist):
    y_test= testdata[4].tolist()
    cm = np.zeros((2,2))
    for i in range(0,len(y_test)):
        rl = y_test[i]
        pre = prelist[i]
        cm[rl-1][pre-1] +=1
    return cm

In [77]:
# Compute the confusion matrix, precision, recall,F-measure, and accuracy
def evalpf(cm):
    tp = cm[0][0]
    fp = cm[0][1]
    fn = cm[1][0]
    tn = cm[1][1]
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    Fm = 2*precision*recall/(precision+recall)
    return (precision,recall,Fm,accuracy)

In [182]:
def clsmain(fn='iris.data', kfold = 5, classnum = 2, shuffle = True):
    data = rf(fn)
    total = len(data)
    i_test, i_train = k_fold(total,kfold,True)
    prelist = []
    data_test =data.iloc[i_test[0],] 
    data_train=data.iloc[i_train[0],]
    parms = getPrmtr(data_train,classnum)  #(mean, sigma/mean, m )
    alfa = parms[2]/total
    y_predict = classificaiton(data_test, alfa, parms[0], parms[1])
    # y_test = data_test[4]
    # print type(y_test)
    cm = cMatrix(data_test,y_predict)
    print 'confusion matrix is '
    print cm
    result = evalpf(cm)
    print 'result of precision, recall,F-measure, and accuracy are'
    print result
  
clsmain()

confusion matrix is 
[[ 10.   0.]
 [  0.  10.]]
result of precision, recall,F-measure, and accuracy are
(1.0, 1.0, 1.0, 1.0)


### This result shows that the membership function classified datas perfectly.